In [4]:
import tensorflow as tf

In [5]:
path_to_file = tf.keras.utils.get_file(
    'shakespeare.txt',
    'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt'
)
with open(path_to_file) as f:
    text = f.read()
    print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [6]:
idx2char = sorted(set(text))
char2idx = {c: i for i, c in enumerate(idx2char)}
list(char2idx.items())[:5]

[('\n', 0), (' ', 1), ('!', 2), ('$', 3), ('&', 4)]

In [7]:
text_as_int = [char2idx[c] for c in text]
text_as_int[:10]

[18, 47, 56, 57, 58, 1, 15, 47, 58, 47]

In [8]:
SEQUENCE_LENGTH, BATCH_SIZE, BUFFER_SIZE = 100, 64, 10_000
dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
dataset = dataset.batch(SEQUENCE_LENGTH + 1, drop_remainder=True)
dataset = dataset.map(lambda s: (s[:-1], s[1:]))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
next(iter(dataset.take(1)))

(<tf.Tensor: id=36, shape=(64, 100), dtype=int32, numpy=
 array([[53, 44,  1, ...,  1, 50, 39],
        [58, 46,  1, ...,  5, 42,  6],
        [41, 49,  6, ...,  1, 30, 21],
        ...,
        [12,  0,  0, ..., 58, 53, 56],
        [56, 50, 39, ..., 53,  1, 54],
        [34, 27, 24, ..., 59, 57,  1]], dtype=int32)>,
 <tf.Tensor: id=37, shape=(64, 100), dtype=int32, numpy=
 array([[44,  1, 44, ..., 50, 39, 64],
        [46,  1, 49, ..., 42,  6,  0],
        [49,  6,  1, ..., 30, 21, 15],
        ...,
        [ 0,  0, 13, ..., 53, 56, 57],
        [50, 39, 52, ...,  1, 54, 59],
        [27, 24, 33, ..., 57,  1, 42]], dtype=int32)>)

In [9]:
L = tf.keras.layers
EMBEDDING_SIZE = 8
RNN_SIZE = 512
def get_model(batch_size=BATCH_SIZE):
    return tf.keras.Sequential([
        L.Embedding(len(idx2char), EMBEDDING_SIZE,
                    batch_input_shape=(batch_size, None, )),
        L.GRU(RNN_SIZE, return_sequences=True, stateful=True),
        L.Dense(len(idx2char)),
    ])
model = get_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 8)             520       
_________________________________________________________________
gru (GRU)                    (64, None, 512)           801792    
_________________________________________________________________
dense (Dense)                (64, None, 65)            33345     
Total params: 835,657
Trainable params: 835,657
Non-trainable params: 0
_________________________________________________________________


In [12]:
batch = next(iter(dataset.take(1)))
predictions = model(batch)
predictions = tf.argmax(predictions, -1)
predictions = predictions[0]
sentence = "".join((idx2char[c.numpy()] for c in batch[0][0]))
prediction = "".join((idx2char[c.numpy()] for c in predictions))
sentence, prediction

(' cousin,\nWhom it concerns to hear this matter forth,\nDo with your injuries as seems you best,\nIn any',
 'Vhultng\nAhi  In iomtest  th tirr thes canter trr h \nAo tith tour snnuryes an toe s aou au t \nA  t y ')

In [14]:
loss_func = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
true = tf.reshape([1], (1, ))
pred = tf.reshape([.8, .1, -3.4], (1, 3))
loss_func(true, pred)

<tf.Tensor: id=2410, shape=(), dtype=float32, numpy=1.113156>

In [16]:
model.compile(optimizer="adam", loss=loss_func,
              metrics=['sparse_categorical_accuracy'])
model.fit(dataset, epochs=100)

Epoch 1/100
172/172 [==============================] - 18s 102ms/step - loss: 1.4963 - sparse_categorical_accuracy: 0.5556
Epoch 2/100
172/172 [==============================] - 17s 98ms/step - loss: 1.4656 - sparse_categorical_accuracy: 0.5639
Epoch 3/100
172/172 [==============================] - 17s 99ms/step - loss: 1.4389 - sparse_categorical_accuracy: 0.5702
Epoch 4/100
172/172 [==============================] - 17s 100ms/step - loss: 1.4172 - sparse_categorical_accuracy: 0.5757
Epoch 5/100
172/172 [==============================] - 17s 100ms/step - loss: 1.3959 - sparse_categorical_accuracy: 0.5813
Epoch 6/100
172/172 [==============================] - 17s 100ms/step - loss: 1.3769 - sparse_categorical_accuracy: 0.5865
Epoch 7/100
172/172 [==============================] - 17s 100ms/step - loss: 1.3603 - sparse_categorical_accuracy: 0.5906
Epoch 8/100
172/172 [==============================] - 17s 98ms/step - loss: 1.3439 - sparse_categorical_accuracy: 0.5954
Epoch 9/100
172/172

172/172 [==============================] - 17s 99ms/step - loss: 0.9674 - sparse_categorical_accuracy: 0.7141
Epoch 62/100
172/172 [==============================] - 17s 99ms/step - loss: 0.9634 - sparse_categorical_accuracy: 0.7153
Epoch 63/100
172/172 [==============================] - 17s 98ms/step - loss: 0.9625 - sparse_categorical_accuracy: 0.7161
Epoch 64/100
172/172 [==============================] - 17s 97ms/step - loss: 0.9621 - sparse_categorical_accuracy: 0.7160
Epoch 65/100
172/172 [==============================] - 17s 98ms/step - loss: 0.9614 - sparse_categorical_accuracy: 0.7163
Epoch 66/100
172/172 [==============================] - 17s 97ms/step - loss: 0.9564 - sparse_categorical_accuracy: 0.7176
Epoch 67/100
172/172 [==============================] - 17s 97ms/step - loss: 0.9565 - sparse_categorical_accuracy: 0.7173
Epoch 68/100
172/172 [==============================] - 17s 98ms/step - loss: 0.9549 - sparse_categorical_accuracy: 0.7181
Epoch 69/100
172/172 [=======

In [35]:
# model = tf.keras.models.load_model('text_generation_backup.h5')
# model.save('text_generation_backup.h5')
# model.load_weights('text_generation.h5')

In [17]:
model.save('text_generation.h5')
model = get_model(1)
model.load_weights('text_generation.h5')
model.summary()
batch = next(iter(dataset.take(1)))
sentence = batch[0][:1]
predictions = model(sentence)
predictions = tf.argmax(predictions, -1)
sentence = "".join((idx2char[i] for i in sentence[0]))
predictions = "".join((idx2char[i] for i in predictions[0]))
sentence, predictions

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 8)              520       
_________________________________________________________________
gru_1 (GRU)                  (1, None, 512)            801792    
_________________________________________________________________
dense_1 (Dense)              (1, None, 65)             33345     
Total params: 835,657
Trainable params: 835,657
Non-trainable params: 0
_________________________________________________________________


("unce it faithfully:\nOr if thou think'st I am too quickly won,\nI'll frown and be perverse an say thee",
 "ndh:In ioirh,ul y:\niu if hhou think'st t am io  fuickly won,\nI'll foiwn and seaaersereedon ahy thee ")

In [34]:
def generate(start_string="ROMEO: ", length=100, temperature=1.):
    start = [char2idx[c] for c in start_string]
    start = tf.reshape(start, (1, -1))
    model.reset_states()
    for _ in range(length):
        prediction = model(start)
        prediction = prediction[:, -1, :]
        # prediction = tf.argmax(predictions, -1)
        prediction = tf.random.categorical(prediction / temperature, 1)
        start = tf.reshape(prediction, (1, 1))
        start_string += idx2char[tf.reshape(prediction, ()).numpy()]
    return start_string
print(generate(length=1000, temperature=.5))

ROMEO: there 'tis here?
I pray thee, more than length.

Servented:
I lay an old report and one that word
'Twixt you there, and that the gods pale,
The precious Plouding one another second bears,
And make me stay with him and left them both.

BAPTISTA:
I have forgot you to your grace:
I hope the crown beloved:
Let him not think there's a fearful house.

KING HENRY VI:
Hark you, sir, the greater side in him adversaries
Persuadest make me to the good worse.

GLOUCESTER:
I shall entreat your knowledge with
statue of your daughter's earth arive away,
That thou canst nothing but my heart to see,
Who good be recond to the people.

COMINIUS:
He's sentenced here the chase of both,
And save your father is changed into the king.

DUKE OF AUMERLE:
Unnew what I should sing it, in the new do me to be so but the ship of men
The great complaining of thy love,
Is not a metse princes. O that bear them
Romoritable and friends no son of my imprisonment.

GLOUCESTER:
The first seven in the contrary doth of